# **Dicionário**

id_ocorrencia - identificação única da ocorrência

cidade - A cidade onde aconteceu a ocorrência

população - Pupulação da cidade

horario - horario da ocorrencia com data e hora

vtr - viatura empenhanha na ocorrencia

mtcl - matricula do militar ou cpf do Bombeiro comunitario

militar_bc - identificação se o individuo é Bombeiro Comunitário ou Bombeiro
Militar

bc_indenizado - Identifica se o Bombeiro Comunitario estava no serviço sendo
pago(indenizado) ou não

obm - Organização Bombeiro Militar que estava na ocorrência

coordenada_obm - Coordenada da organização Bombeiro Militar

coordenada_ocorrencia - Coordenada da Ocorrência

lat_ocorrencia - Latitude ocorrência

lon_ocorrencia - Longitude Ocorrência

tipo - Tipo da ocorrência, pode ser Atendimento pré-hospitalar - APH, incendio, acidente e outros.

subtipo - especifica o tipo, num APH, pode especificar desmaio, nem sempre é preenchido.

tempo_resposta - Tempo do acionamento até a chegada ao local da ocorrêcia, pode ser um dado que estaja bem falho, pois acontece do bombeiro esquecer de clicar no botão

tempo_total_empenho - tempo total da saída da obm até a chegada a obm novamente.
firecast - indica se uma viatura utilizou o sistema "firecast" para dar J-9, J-10, J-11 e J-12 nos deslocamentos

origem_samu - Se foi um repasse da ocorrência que era do samu

vitima - geração no sistema, talvez de pra gerar o número de vítimas, nome, idade, alguns sinais vitais.

In [2]:
from sklearn.impute import KNNImputer
import pandas as pd
import os
from datetime import date

df = pd.read_csv('Data/dataset_ocorrencias.csv', sep=';', thousands=',')

X = df.loc[:, ['id_ocorrencia', 'cidade', 'horario', 'lat_ocorrencia', 'lon_ocorrencia', 'tipo']]
X = X.loc[X['cidade'] == 'Florianópolis']
X = X.loc[X['tipo'] == 'ATENDIMENTO PRÉ-HOSPITALAR']
X = X.loc[X["lon_ocorrencia"] >= -48.6, :]

# Create a copy of the DataFrame with only the columns of interest
impute_df = X[['lat_ocorrencia', 'lon_ocorrencia']].copy()

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=5)  # Choose the appropriate number of neighbors

# Perform KNN imputation
imputed_values = imputer.fit_transform(impute_df)

# Replace the columns with the imputed values
X['lat_ocorrencia'] = imputed_values[:, 0]
X['lon_ocorrencia'] = imputed_values[:, 1]

# Drop duplicates since id_ocorrencia is unique
X = X.drop_duplicates(subset="id_ocorrencia", keep='first')
X["Date"] = pd.to_datetime(X["horario"], infer_datetime_format=True).dt.floor('1D')

# Filter data for a specific date if needed
# X = X.loc[X['Date'] >= '2019-07-01']

# Save DataFrame to CSV with the current date in the filename
output_directory = "Data/datasets"
os.makedirs(output_directory, exist_ok=True)
current_date = date.today().strftime("%Y-%m-%d")
output_filename = f"dataset_{current_date}.csv"
output_path = os.path.join(output_directory, output_filename)
X.to_csv(output_path, index=False)

print(f"Dataset saved to: {output_path}")

/tmp/ipykernel_344954/3334746939.py:6: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/dataset_ocorrencias.csv', sep=';', thousands=',')


Dataset saved to: Data/datasets/dataset_2023-07-16.csv
